# This notebook provides interactive widgets to play with the imaging requirements for the SKA1 SDP

## First, do the necessary set-up (run this code block first)

In [1]:
'''
The following commands import the python modules, methods and code fragments that are required to run this notebook. 
Please refer to the following python files for implementation details (omitted here for readability)
* env_setup.py -- code for setting up the python environment
* parameter_definitions.py -- contains definitions of variables, primary telescope parameters  
* formulae.py -- contains formulae for defiving secondary telescope-specific parameters from input parameters
* implementation.py -- contains methods for performing computations (i.e. crunching the numbers)
'''

from IPython.html.widgets import interact, interactive, fixed
from IPython.html.widgets.interaction import interact_manual
from IPython.html import widgets
from IPython.display import clear_output, display, HTML

from env_setup import *
from parameter_definitions import *
from formulae import *
from implementation import Implementation as imp

def show_table(header, titles, values, units):
    s = '<h3>%s:</h3><table>\n' % header
    assert len(titles) == len(values)
    assert len(titles) == len(units)
    for i in range(len(titles)):
        s += '<tr><td>{0}</td><td><font color="blue">{1}</font> {2}</td></tr>\n'.format(titles[i], values[i], units[i])
    s += '</table>'
    display(HTML(s))

band_lookup = {'SKA1-Low':'Low', 'SKA1-Mid (Band 1)':'Mid1', 'SKA1-Survey (Band 1)':'Sur1'}
telescopes_pretty_print =('SKA1-Low', 'SKA1-Mid (Band 1)', 'SKA1-Survey (Band 1)')
modes_pretty_print = ('Continuum', 'Spectral', 'SlowTrans')

mode_lookup = {}
for key in modes_pretty_print:
    mode_lookup[key] = key

def evaluate_expressions(expressions, tp, Tsnap, Nfacet):
    """
    Evaluate a sequence of expressions by substituting the telescope_parameters into them. Returns the result
    """    
    results = []
    for expression in expressions:
        expression_subst = -1
        try:
            expression_subst = expression.subs({tp.Tsnap : Tsnap, tp.Nfacet : Nfacet})
            result = imp.evaluate_binned_expression(expression_subst, tp)
        except Exception as e:
            result = expression
            
        results.append(result)
    return results

def compute_flops_subset(max_baseline, Nf_max, Nfacet, Tsnap, Telescope, Mode):    
    # First we plot a table with all the provided parameters
    band = band_lookup[Telescope]
    param_titles = ('Max Baseline', 'Max # of channels', 'Telescope', 'Band', 'Mode', 'Tsnap', 'Nfacet')
    param_values = (max_baseline, Nf_max, Telescope, band, Mode, Tsnap, Nfacet)
    param_units = ('km', '', '', '', '', 'sec', '')
    show_table('Arguments', param_titles, param_values, param_units)
    
    # And now the results:
    tp = imp.calc_tel_params(band=band, mode=mode_lookup[Mode])  # Calculate the telescope parameters        
    max_allowed_baseline = tp.baseline_bins[-1] / u.km    
    if max_baseline <= max_allowed_baseline:
        tp.Bmax = max_baseline * u.km
        tp.Nf_max = Nf_max
        imp.update_derived_parameters(tp, mode=mode_lookup[Mode])
        # The result expressions need to be defined here as they depend on tp (updated in the line above)
        result_expressions = (tp.Mbuf_vis/u.peta, tp.Mw_cache/u.tera, tp.Npix_linear, tp.Rio/u.tera, 
                              tp.Rflop/u.peta)
        result_titles = ('Visibility Buffer', 'Working (cache) memory', 'Image side length', 'I/O Rate', 
                         'Total Compute Requirement')
        result_units = ('PetaBytes', 'TeraBytes', 'pixels', 'TeraBytes/s','PetaFLOPS')
        result_values = evaluate_expressions(result_expressions, tp, Tsnap, Nfacet)
        result_value_string = []
        for i in range(len(result_values)):
            expression = result_expressions[i]
            if expression is not tp.Npix_linear: 
                result_value_string.append('%.3g' % result_values[i])
            else:
                result_value_string.append('%d' % result_values[i])

        show_table('Computed Values', result_titles, result_value_string, result_units)
    else :
        msg = 'ERROR: max_baseline exceeds the maximum allowed baseline of %g km for this telescope.' % max_allowed_baseline
        s = '<font color="red"><b>{0}</b>.<br>Adjust to recompute.</font>'.format(msg)
        display(HTML(s))

def compute_flops_full(max_baseline, Nf_max, Telescope, Mode):
    # First we plot a table with all the provided parameters
    band = band_lookup[Telescope]
    param_titles = ('Max Baseline', 'Max # of channels', 'Telescope', 'Band', 'Mode')
    param_values = (max_baseline, Nf_max, Telescope, band, Mode)
    param_units = ('km', '', '', '', '')
    show_table('Arguments', param_titles, param_values, param_units)
    
    # And now the results:
    display(HTML('<font color="blue">Computing the result -- this may take several (tens of) seconds.</font>'))
    tp = imp.calc_tel_params(band=band, mode=mode_lookup[Mode])  # Calculate the telescope parameters        
    max_allowed_baseline = tp.baseline_bins[-1] / u.km    
    if max_baseline <= max_allowed_baseline:
        tp.Bmax = max_baseline * u.km
        tp.Nf_max = Nf_max
        imp.update_derived_parameters(tp, mode=mode_lookup[Mode])
        (Tsnap, Nfacet) = imp.find_optimal_Tsnap_Nfacet(tp, verbose=True)
        
        # The result expressions need to be defined here as they depend on tp (updated in the line above)
        result_expressions = (tp.Mbuf_vis/u.peta, tp.Mw_cache/u.tera, tp.Npix_linear, tp.Rio/u.tera, 
                              tp.Rflop/u.peta)
        result_titles = ('Number of Facets', 'Snapshot Time', 'Visibility Buffer', 'Working (cache) memory', 
                         'Image side length', 'I/O Rate', 'Total Compute Requirement',)
        result_units = ('', 'sec.', 'PetaBytes', 'TeraBytes', 'pixels', 'TeraBytes/s','PetaFLOPS',)
        
        result_value_string = []
        result_value_string.append('%d' % Nfacet)
        result_value_string.append('%.3g' % Tsnap)
        result_values = evaluate_expressions(result_expressions, tp, Tsnap, Nfacet)
        for i in range(len(result_values)):
            expression = result_expressions[i]
            if expression is not tp.Npix_linear: 
                result_value_string.append('%.3g' % result_values[i])
            else:
                result_value_string.append('%d' % result_values[i])

        show_table('Computed Values', result_titles, result_value_string, result_units)
    else :
        msg = 'ERROR: max_baseline exceeds the maximum allowed baseline of %g km for this telescope.' % max_allowed_baseline
        s = '<font color="red"><b>{0}</b>.<br>Adjust to recompute.</font>'.format(msg)
        display(HTML(s))

IPython console for SymPy 0.7.6 (Python 2.7.7-32-bit) (ground types: python)


:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pylab as pylab

def plot_flops(title, rflop_grid, rflop_fft, rflop_proj, rflop_conv):
    pylab.rcParams['figure.figsize'] = 8, 6  # that's default image size for this interactive session

    values = results_per_tel[tel]

    # The slices will be ordered and plotted counter-clockwise.
    labels = '(de)Gridding', '(i)FFT', '(Re)Projection', 'Convolution'
    sizes = [rflop_grid, rflop_fft, rflop_proj, rflop_conv]
    colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
    explode = (0.05, 0.05, 0.05, 0.05) # Radial offset of the slices

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    # Set aspect ratio to be equal so that pie is drawn as a circle.
    plt.axis('equal')
    plt.title('%s\n' % title)

    plt.show()

## Now, we can interactively play with parameters

### The first option is automatic updating of results as the sliders are moved. This may be sluggish

In [2]:
interact(compute_flops_subset, max_baseline=(10,200), Nf_max = (1,256000,1), Nfacet=(1,10,1), Tsnap=(1.2,1800), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,88 km
Max # of channels,128000
Telescope,SKA1-Low
Band,Low
Mode,Continuum
Tsnap,78.5 sec
Nfacet,3


Visibility Buffer,1 PetaBytes
Working (cache) memory,0.0418 TeraBytes
Image side length,110348 pixels
I/O Rate,2.3 TeraBytes/s
Total Compute Requirement,2.86 PetaFLOPS


### The second option is manual triggering of recompute events (recommended). 

**This allows more conveniently computing elaborate (slow) optimizations and visualizations per computation, as these are only run when required**

In [3]:
# In this example, Tsnap and Nfacet are *automatically* chosen so as to minimize the value of Rflop

interact_manual(compute_flops_full, max_baseline=(10,200), Nf_max = (1,256000,1), Telescope=telescopes_pretty_print, 
         Mode=modes_pretty_print);

Max Baseline,88 km
Max # of channels,128000
Telescope,SKA1-Low
Band,Low
Mode,Continuum


Evaluating Nfacets = 1
Tsnap has been optimized as : 33.520121, yielding a minimum value of 4.355135 Peta-units
Evaluating Nfacets = 2
Tsnap has been optimized as : 53.223321, yielding a minimum value of 2.949260 Peta-units
Evaluating Nfacets = 3
Tsnap has been optimized as : 75.169219, yielding a minimum value of 2.854483 Peta-units
Evaluating Nfacets = 4
Tsnap has been optimized as : 95.490156, yielding a minimum value of 3.036833 Peta-units

Expression increasing with number of facets; aborting exploration of Nfacets > 4

2.854483 PetaFLOPS was the lowest FLOP value, found for (Nfacet, Tsnap) = (3, 75.17)
(7, 7, 7)


Number of Facets,3
Snapshot Time,75.2 sec.
Visibility Buffer,1 PetaBytes
Working (cache) memory,0.0379 TeraBytes
Image side length,110348 pixels
I/O Rate,2.3 TeraBytes/s
Total Compute Requirement,2.85 PetaFLOPS
